In [16]:
''' Outdated models '''
# MODEL_NAME='lucyd-edof.pth'
# MODEL_NAME='lucyd-edof-11.pth'

''' New/relevant models '''
# MODEL_NAME='lucyd-small-phantoms.pth' # original model trained on small phantoms
# MODEL_NAME='lucyd-psf-sim.pth' # improved EDOF blur simlation (3d projections + microscopy PSF simulation)
MODEL_NAME='lucyd-psf-sim-2.pth' # improved EDOF blur simlation (slightly less patchy results, trained with finer structures)

In [17]:
from lucyd import LUCYD, device
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

import torch
import time 
import os

import cv2 as cv
import numpy as np

Parameters:

In [19]:
RESIZE=True
RESIZE_RESOLUTION=(2560,2560) # will be ignored if RESIZE=False
BG_CORR = True

model = LUCYD(num_res=1).to(device)
model.load_state_dict(torch.load('models/'+MODEL_NAME))
model.eval()
print('number of parameters: {}'.format(sum(p.numel() for p in model.parameters())))

path = '/mnt/m181 PISCO1/M181-175-1_CTD-050_00deg00S-019deg00W_20220509-0543/PNG/'

number of parameters: 8452


Background correction:

In [4]:
if BG_CORR:
    files = os.listdir(path)
    imgs = [cv.imread(os.path.join(path, file), cv.IMREAD_GRAYSCALE) for file in files]
    bg = np.max(imgs, axis=0)
    corrected = cv.absdiff(imgs[0], bg)
else:
    pass

KeyboardInterrupt: 

In [20]:
################
# NAME OF THE IMAGE YOU WANT TO DECONVOLVE
path = '/mnt/m181 PISCO1/M181-175-1_CTD-050_00deg00S-019deg00W_20220509-0543/PNG/'
for file in os.listdir(path):

    filename = os.path.join(path,file)
    result_fn = os.path.join('/home/plankton/Results/M181/Deconv/M181-175-1_CTD-050_00deg00S-019deg00W_20220509-0543',file)
    ################

    with torch.no_grad():
        if RESIZE:
            x = np.array(Image.open(filename).convert('L').resize(RESIZE_RESOLUTION))
        else:
            x = np.array(Image.open(filename).convert('L'))

        if x.shape[0] % 2 != 0: x = x[1:]
        if x.shape[1] % 2 != 0: x = x[:,1:]
        # x = (x - np.min(x))/(np.max(x) - np.min(x))
        x = x/255

        x_torch = torch.from_numpy(x).to(device)
        x_torch = x_torch.unsqueeze(0).unsqueeze(0)

        start_time = time.time()
        y_hat, y_k, update = model(x_torch.float())
        print('Time to deconvolve: {} seconds'.format(time.time() - start_time))

        # plt.figure(figsize=(20,20))
        # plt.subplot(1,2,1)
        y = x_torch.detach().cpu().numpy()[0,0]
        # plt.imshow(y, cmap='gray')
        # plt.title('Input')
        # plt.subplot(1,2,2)
        x = y_hat.detach().cpu().numpy()[0,0]
        # plt.imshow(x, cmap='gray', vmin=0)
        # plt.title('Output')
        # plt.show()

        Image.fromarray(x*255.).convert('L').save(result_fn)

Time to deconvolve: 0.15438199043273926 seconds
Time to deconvolve: 0.0073854923248291016 seconds
Time to deconvolve: 0.004471540451049805 seconds
Time to deconvolve: 0.005179882049560547 seconds
Time to deconvolve: 0.005849123001098633 seconds
Time to deconvolve: 0.004902839660644531 seconds
Time to deconvolve: 0.005044698715209961 seconds
Time to deconvolve: 0.006053924560546875 seconds
Time to deconvolve: 0.004500865936279297 seconds
Time to deconvolve: 0.004382133483886719 seconds
Time to deconvolve: 0.0056531429290771484 seconds
Time to deconvolve: 0.0060672760009765625 seconds
Time to deconvolve: 0.008312463760375977 seconds
Time to deconvolve: 0.004929542541503906 seconds
Time to deconvolve: 0.0050885677337646484 seconds
Time to deconvolve: 0.004903316497802734 seconds
Time to deconvolve: 0.004441499710083008 seconds
Time to deconvolve: 0.005251407623291016 seconds
Time to deconvolve: 0.004329204559326172 seconds
Time to deconvolve: 0.0056688785552978516 seconds
Time to deconvol

In [27]:
#!cd /home/plankton/PISCO_Software/MaxSegmenter
from MaxSegmenterModule import run_segmenter

run_segmenter(
        #source_folder="C:/Users/timka/Documents/Arbeit/PeruTest",
        source_folder="/home/plankton/Results/M181/Deconv/M181-175-1_CTD-050_00deg00S-019deg00W_20220509-0543/",
        save_crops=False,
        save_marked_imgs=False,
        save_bg_corr_imgs=True,
        min_area_to_segment=1000,
        min_area_to_save=1000,
        save_path="/home/plankton/Results/M181/Deconv/M181-175-1_CTD-050_00deg00S-019deg00W_20220509-0543/bg_corr/",
        #save_path="C:/Users/timka/Documents/Arbeit/Results",
        equalize_hist=True,
        resize=True,
        clear_save_path=True,
        bg_size=5,
        max_threads=10,
        n_sigma=2, #2 for M181 images, 1.5 for Cusco Peru
        n_cores=8,
        mask_imgs=True,
        mask_radius_offset=200,
    )

unprocessed directory found!
remaining images: 20522
Start segmenting
Mask radius is: 1302
10% done
30% done
40% done
50% done
70% done
80% done
100% done
Total time: 1610.5983849840704, Avg time per img: 0.07848155077400207


In [26]:
%cd /home/plankton/PISCO_Software/MaxSegmenter
%pwd

/home/plankton/PISCO_Software/MaxSegmenter


'/home/plankton/PISCO_Software/MaxSegmenter'

In [28]:
run_segmenter(
        #source_folder="C:/Users/timka/Documents/Arbeit/PeruTest",
        source_folder="/mnt/m181 PISCO1/M181-175-1_CTD-050_00deg00S-019deg00W_20220509-0543/PNG/",
        save_crops=False,
        save_marked_imgs=False,
        save_bg_corr_imgs=True,
        min_area_to_segment=1000,
        min_area_to_save=1000,
        save_path="/home/plankton/Results/M181/M181-175-1_CTD-050_00deg00S-019deg00W_20220509-0543/",
        #save_path="C:/Users/timka/Documents/Arbeit/Results",
        equalize_hist=True,
        resize=True,
        clear_save_path=True,
        bg_size=5,
        max_threads=10,
        n_sigma=2, #2 for M181 images, 1.5 for Cusco Peru
        n_cores=8,
        mask_imgs=True,
        mask_radius_offset=200,
    )

unprocessed directory found!
remaining images: 20521
Start segmenting
Mask radius is: 2820
No progress detected, terminating status update...
Total time: 2011.0037388849305, Avg time per img: 0.09799735582500514


In [30]:
################
# NAME OF THE IMAGE YOU WANT TO DECONVOLVE
path = '/home/plankton/Results/M181/M181-175-1_CTD-050_00deg00S-019deg00W_20220509-0543/Images/'
for file in os.listdir(path):

    filename = os.path.join(path,file)
    result_fn = os.path.join('/home/plankton/Results/M181/Deconv/M181-175-1_CTD-050_00deg00S-019deg00W_20220509-0543_bg_corr',file)
    ################

    with torch.no_grad():
        if RESIZE:
            x = np.array(Image.open(filename).convert('L').resize(RESIZE_RESOLUTION))
        else:
            x = np.array(Image.open(filename).convert('L'))

        if x.shape[0] % 2 != 0: x = x[1:]
        if x.shape[1] % 2 != 0: x = x[:,1:]
        # x = (x - np.min(x))/(np.max(x) - np.min(x))
        x = x/255

        x_torch = torch.from_numpy(x).to(device)
        x_torch = x_torch.unsqueeze(0).unsqueeze(0)

        start_time = time.time()
        y_hat, y_k, update = model(x_torch.float())
        print('Time to deconvolve: {} seconds'.format(time.time() - start_time))

        # plt.figure(figsize=(20,20))
        # plt.subplot(1,2,1)
        y = x_torch.detach().cpu().numpy()[0,0]
        # plt.imshow(y, cmap='gray')
        # plt.title('Input')
        # plt.subplot(1,2,2)
        x = y_hat.detach().cpu().numpy()[0,0]
        # plt.imshow(x, cmap='gray', vmin=0)
        # plt.title('Output')
        # plt.show()

        Image.fromarray(x*255.).convert('L').save(result_fn)

Time to deconvolve: 0.11010146141052246 seconds
Time to deconvolve: 0.007553815841674805 seconds
Time to deconvolve: 0.00644230842590332 seconds
Time to deconvolve: 0.007519721984863281 seconds
Time to deconvolve: 0.005121707916259766 seconds
Time to deconvolve: 0.006467103958129883 seconds
Time to deconvolve: 0.007014036178588867 seconds
Time to deconvolve: 0.013376712799072266 seconds
Time to deconvolve: 0.006436824798583984 seconds
Time to deconvolve: 0.0051381587982177734 seconds
Time to deconvolve: 0.005406618118286133 seconds
Time to deconvolve: 0.005468845367431641 seconds
Time to deconvolve: 0.0069620609283447266 seconds
Time to deconvolve: 0.005045890808105469 seconds
Time to deconvolve: 0.00690913200378418 seconds
Time to deconvolve: 0.006078958511352539 seconds
Time to deconvolve: 0.007961511611938477 seconds
Time to deconvolve: 0.005834341049194336 seconds
Time to deconvolve: 0.00554203987121582 seconds
Time to deconvolve: 0.0061244964599609375 seconds
Time to deconvolve: 0